In [1]:
"""
Simple tester for the deep3d
"""

import tensorflow as tf
import vgg19_trainable as vgg19
import utils
import numpy as np
import os
import random
import os.path
import h5py
import matplotlib as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
#dataset will be 224 x 224 crops out of the inria dataset, with random labels from 1 to 10

inria_file = '/a/data/deep3d_data/inria_data.h5'
h5f = h5py.File(inria_file,'r')

X_train = np.zeros((400,224,224,3))
Y_train = np.random.randint(10,size=(400,1000))
X_val = np.zeros((50,224,224,3))
Y_val = np.random.randint(10,size=50)
X_train[:,22:202,:,:] = h5f['X_0'][:400,:,48:272,:]
X_val[:,22:202,:,:] = h5f['X_0'][400:450,:,48:272,:]

h5f.close()

print Y_train.shape

(400, 1000)


In [3]:
num_batches = 20
batchsize = 50
device1 = '/gpu:0'
#device1 = '/cpu:0'

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction=1

with tf.device(device1):
    sess = tf.Session(config=config)

    images = tf.placeholder(tf.float32, [batchsize, 224, 224, 3])
    true_out = tf.placeholder(tf.float32, [batchsize, 1000])
    
    train_mode = tf.placeholder(tf.bool)

    net = vgg19.Vgg19('./vgg19.npy')
    net.build(images, train_mode)

    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print 'var count:'
    print(net.get_var_count())

    sess.run(tf.global_variables_initializer())

    # simple 1-step training
    cost = tf.reduce_sum((net.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.002).minimize(cost)
    for i in xrange(num_batches):
        image_mask = np.random.choice(X_train.shape[0],batchsize)
        images_in = X_train[image_mask,:,:,:]
        labels_in = Y_train[image_mask,:]
        _, cost_val = sess.run([train,cost], feed_dict={images: images_in, true_out: labels_in, train_mode: True})
        print 'cost:'
        print cost_val
 

[224, 224, 1]
var count:
143667240
